## Import liabraries

In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from skimage import data,color,exposure
import cv2
import matplotlib.pyplot as plt
import os
import cv2
from sklearn import svm
from sklearn.model_selection import train_test_split
from skimage import color
import numpy as np
import pandas as pd
import joblib

In [ ]:
# Import liabraries


def extract_features(image):
    
    hist = cv2.calcHist([image], [0], None, [256], [0, 256])
    hist = hist.flatten()

    edges = cv2.Canny(image, 100, 200)
    edges_sum = np.sum(edges)

    features = np.hstack((hist, edges_sum))
    return features

## Load and preprocess the data

def load_data(data_dir):
    images = []
    labels = []
    for label in os.listdir(data_dir):
        class_dir = os.path.join(data_dir, label)
        if os.path.isdir(class_dir):
            for file in os.listdir(class_dir):
                file_path = os.path.join(class_dir, file)
                
                image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
                if image is not None:
                    features = extract_features(image)
                    images.append(features)
                    labels.append(label)
    return np.array(images), np.array(labels)

data_dir = 'dataset_full'
X, y = load_data(data_dir)

print(X, y)

# Split data to train and test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Train the SVM model

svm_model = SVC(kernel='linear', probability=True)
svm_model.fit(X_train, y_train)

# Evaluate the model

y_pred = svm_model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Save the model

joblib.dump(svm_model, 'svm_model.pkl')


## Flask app to upload the image and classify it

In [ ]:
# Flask app


from flask import Flask, request, jsonify, render_template
import joblib
import cv2

app = Flask(__name__)
model = joblib.load('model.pkl') 

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    file = request.files['file']
    img = cv2.imdecode(np.frombuffer(file.read(), np.uint8), cv2.IMREAD_GRAYSCALE)
    features = extract_features(img) 
    prediction = model.predict([features])
    return jsonify({'prediction': int(prediction[0])})

if __name__ == "__main__":
    app.run(debug=True)
